In [2]:
# Paste this into a Jupyter notebook cell and run.
import json, time, os
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def load_cookies_from_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Accept either a dict with "cookies" key or a plain list
    if isinstance(data, dict) and "cookies" in data:
        return data["cookies"]
    return data

def normalize_cookie(raw):
    # Build cookie dict accepted by selenium (only allowed keys)
    cookie = {}
    # required
    cookie['name'] = raw.get('name') or raw.get('Name') or raw.get('key')
    cookie['value'] = raw.get('value') or raw.get('Value')
    if not cookie['name'] or cookie['value'] is None:
        raise ValueError("Cookie missing name or value: {}".format(raw))
    # optional accepted fields
    if raw.get('domain') is not None:
        cookie['domain'] = raw.get('domain')
    if raw.get('path') is not None:
        cookie['path'] = raw.get('path')
    if raw.get('secure') is not None:
        # Selenium expects a boolean
        cookie['secure'] = bool(raw.get('secure'))
    if raw.get('httpOnly') is not None:
        cookie['httpOnly'] = bool(raw.get('httpOnly'))
    # expiry may be provided as int or string; Selenium expects int (seconds since epoch)
    exp = raw.get('expiry') or raw.get('expires') or raw.get('expirationDate')
    if exp is not None:
        try:
            cookie['expiry'] = int(float(exp))
        except Exception:
            # sometimes expires is a date string; ignore in that case
            pass
    return cookie

# --- Inputs (interactive) ---
target_url = input("Enter full target URL (e.g. https://www.linkedin.com/): ").strip()
cookie_path = input("Enter path to cookie.json (e.g. /path/to/cookie.json): ").strip()

if not target_url:
    raise SystemExit("No URL provided.")
if not cookie_path or not os.path.exists(cookie_path):
    raise SystemExit("cookie.json path not found: {}".format(cookie_path))

# --- Parse domain root ---
parsed = urlparse(target_url)
if not parsed.scheme:
    raise SystemExit("Please include scheme (https://) in the URL.")
domain_root = f"{parsed.scheme}://{parsed.netloc}/"

print(f"Using domain root: {domain_root}")
print(f"Loading cookies from: {cookie_path}")

# --- Load cookies ---
raw_cookies = load_cookies_from_file(cookie_path)
if not isinstance(raw_cookies, list):
    raise SystemExit("cookie.json did not contain a list of cookies or {'cookies': [...]}.")
print(f"Loaded {len(raw_cookies)} cookies (raw).")

# --- Start webdriver ---
options = webdriver.ChromeOptions()
# Uncomment next two lines if you want headless (no browser window)
# options.add_argument("--headless=new")
# options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()
try:
    # go to domain root so cookies can be added for that domain
    driver.get(domain_root)
    time.sleep(1)  # small wait so page context is ready

    # clear existing cookies if you want a clean slate
    driver.delete_all_cookies()
    added = 0
    for raw in raw_cookies:
        try:
            cookie = normalize_cookie(raw)
            # Important: Selenium won't accept cookies with leading dot in domain in some versions,
            # but if cookie['domain'] exists and starts with '.', keep it — Selenium accepts it.
            driver.add_cookie(cookie)
            added += 1
        except Exception as e:
            print("Skipping cookie due to error:", e)

    print(f"Added {added} cookies to the browser session.")
    # Now navigate to the target URL
    driver.get(target_url)
    time.sleep(2)  # give the page a moment to load with injected cookies
    print("Final URL:", driver.current_url)
    print("Page title:", driver.title)

    # If you'd like, you can dump current cookies to verify:
    current = driver.get_cookies()
    print(f"Browser now has {len(current)} cookies (sample):")
    for c in current[:8]:
        print(" -", c.get('name'), "->", c.get('value')[:10] + ("..." if len(c.get('value',''))>10 else ""))

finally:
    # keep browser open if you want to interact manually; comment out quit() if so.
    # driver.quit()
    pass

# End of cell.


Using domain root: https://www.linkedin.com/
Loading cookies from: D:\WebToide\cookies\linkedin_cookies.json
Loaded 21 cookies (raw).
Added 21 cookies to the browser session.
Final URL: https://www.linkedin.com/search/results/people/?origin=MEMBER_PROFILE_CANNED_SEARCH&network=%5B%22F%22%2C%22S%22%5D&connectionOf=%5B%22ACoAAETn0kAB01LKN8u61l2rHc-VxgqJSCX0f-U%22%5D
Page title: Search | LinkedIn
Browser now has 25 cookies (sample):
 - lidc -> "b=OGST05:...
 - sdui_ver -> sdui-flags...
 - AMCV_14215E3D5995C57C0A495C55%40AdobeOrg -> -637568504...
 - liap -> true
 - lang -> v=2&lang=e...
 - JSESSIONID -> "ajax:0308...
 - li_sugr -> 4ab6d326-2...
 - fptctx2 -> taBcrIH61P...
